In [129]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [116]:
df = load_digits()
X = df.data
Y = df.target
Y = np.eye(10)[Y] # one hot

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1437, 64), (360, 64), (1437, 10), (360, 10))

In [117]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

def softmax(X):
    return np.exp(X) / np.sum(np.exp(X))

def root_mean_squared_error(y_gt, y_pred):
    return np.sqrt(np.mean((y_gt - y_pred) ** 2))

In [118]:
epochs = 80
lr = .001 

D_in = X_train.shape[1] # input count (64 input)
H1 = 128 # Hidden layer 1
H2 = 32 # Hidden layer 2
D_out = y_train.shape[1] # 10 class

In [119]:
w1 = np.random.randn(D_in, H1)
w2 = np.random.randn(H1,H2)
w3 = np.random.randn(H2,D_out)

In [120]:
B1 = np.random.randn(1, H1)
B2 = np.random.randn(1, H2)
B3 = np.random.randn(1, D_out)

In [133]:
def precision_recall(y_true, y_pred, num_classes):
    
    for cls in range(num_classes):
        TP = sum((y_true == cls) & (y_pred == cls))  # True Positives
        FP = sum((y_true != cls) & (y_pred == cls))  # False Positives
        FN = sum((y_true == cls) & (y_pred != cls))  # False Negatives
        
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    return {'Precision': precision, 'Recall': recall}

In [125]:
for epoch in range(epochs):
    Y_pred_train = []

    # train
    for x, y in zip(X_train, y_train):
        x = x.reshape(-1, 1)
        # forward 
        ## Layer 1
        layer1_out = sigmoid(x.T @ w1 + B1)
        ## Layer 2
        layer2_out = sigmoid(layer1_out @ w2 + B2)
        ## Layer 3
        layer3_out = softmax(layer2_out @ w3 + B3) 
        y_pred = layer3_out 
        Y_pred_train.append(y_pred)

        
    


        # backpropagation - باید مشتق بگیریم
        ## Layer 3
        error = -2 * (y - y_pred)
        grad_B3 = error
        grad_w3 = error * layer2_out.T
        ## Layer 2
        error = error @ w3.T * layer2_out * (1 - layer2_out)
        grad_B2 = error
        grad_w2 = layer1_out.T @ error 
        ## Layer 1
        error = error @ w2.T * layer1_out * (1 - layer1_out)
        grad_B1 = error
        grad_w1 = x @ error  


        # update
        ## Layer 1
        w1 -= lr * grad_w1
        B1 -= lr * grad_B1
        ## Layer 3
        w2 -= lr * grad_w2
        B2 -= lr * grad_B2
        ## Layer 3
        w3 -= lr * grad_w3
        B3 -= lr * grad_B3


        # acc = ...
        # loss = ...
    
    Y_pred_test = []
    for x, y in zip(X_test, y_test):
        x = x.reshape(-1, 1)
        # forward 
        ## Layer 1
        layer1_out = sigmoid(x.T @ w1 + B1)
        ## Layer 2
        layer2_out = sigmoid(layer1_out @ w2 + B2)
        ## Layer 3
        layer3_out = softmax(layer2_out @ w3 + B3) 
        y_pred = layer3_out 
        Y_pred_test.append(y_pred)

   
    
    Y_pred_test = np.array(Y_pred_test).reshape(-1, 10)
    y_pred_labels = np.argmax(Y_pred_test, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    

    loss_test = root_mean_squared_error(y_test, Y_pred_test)
    accuracy_test = np.sum(np.argmax(y_test, axis= 1) == np.argmax(Y_pred_test, axis=1)) / len(y_test)
    print(accuracy_test)



0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9388888888888889
0.9416666666666667
0.9416666666666667
0.9416666666666667
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9472222222222222
0.9472222222

In [137]:
metrics = precision_recall(y_test_labels, y_pred_labels, num_classes=10)
print(metrics)

precision = precision_score(y_test_labels, y_pred_labels, average='macro')
recall = recall_score(y_test_labels, y_pred_labels, average='macro')
print(f"Precision Sklearn: {precision}\nRecall Sklearn: {recall}")

{'Precision': 0.9393939393939394, 'Recall': 0.8857142857142857}
Precision Sklearn: 0.9424267997497738
Recall Sklearn: 0.9413497306956609


In [ ]:
# import cv2

# img = cv2.imread('input/seven.png')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img = img.reshape(64,1)

# x = img


# # forward 
# ## Layer 1
# layer1_out = sigmoid(x.T @ w1 + B1)
# ## Layer 2
# layer2_out = sigmoid(layer1_out @ w2 + B2)
# ## Layer 3
# layer3_out = softmax(layer2_out @ w3 + B3) 
# y_pred = layer3_out 
# print(np.argmax(y_pred))
